<a href="https://colab.research.google.com/github/TinaKhatri28/ASTRA-Project/blob/main/finalpipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ultralytics==8.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import io
from PIL import Image

# ✅ Load the models
model1 = load_model("/content/drive/MyDrive/sih_project/model1_final.h5")
model2 = load_model("/content/drive/MyDrive/sih_project/model2_final_cotton_disease.h5")
model3 = load_model("/content/drive/MyDrive/sih_project/model3_final_severity.h5")

MODEL1_CLASSES = {0: "healthy", 1: "infected"}
MODEL2_CLASSES = {
    0: "aphids disease",
    1: "armyworm disease",
    2: "bacterial blight disease",
    3: "powdery mildew disease",
    4: "target spot disease"
}
MODEL3_CLASSES = {0: "mild", 1: "moderate", 2: "severe"}


In [ ]:
def crop_leaf(img_cv):
    hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)

    # ✅ Green color range (best for cotton leaves)
    lower_green = np.array([25, 40, 20])
    upper_green = np.array([95, 255, 255])

    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Clean mask
    mask = cv2.medianBlur(mask, 7)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        print("❌ No leaf detected. Make sure leaf is visible & green.")
        return None

    # Largest green object = leaf
    cnt = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)

    if w*h < 5000:
        print("❌ Leaf area too small, move leaf closer.")
        return None

    # Crop with padding
    pad = 20
    x1 = max(0, x - pad)
    y1 = max(0, y - pad)
    x2 = min(img_cv.shape[1], x + w + pad)
    y2 = min(img_cv.shape[0], y + h + pad)

    crop = img_cv[y1:y2, x1:x2]
    return crop


In [ ]:
def preprocess_leaf(img_cv):
    img = cv2.resize(img_cv, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)
    return img


In [ ]:
def pipeline_predict(img_bytes):
    # Convert bytes to OpenCV image
    np_arr = np.frombuffer(img_bytes, np.uint8)
    img_cv = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    leaf = crop_leaf(img_cv)
    if leaf is None:
        print("❌ Pipeline stopped: Leaf not detected.")
        return

    print("✅ Leaf Detected Successfully!")

    # Preprocess
    img = preprocess_leaf(leaf)

    # ✅ Model 1
    p1 = model1.predict(img, verbose=0)[0][0]
    class1 = "healthy" if p1 < 0.5 else "infected"
    print(f"Model 1 = {class1}")

    if class1 == "healthy":
        print("\n✅ FINAL RESULT: HEALTHY LEAF ✅")
        return

    # ✅ Model 2
    p2 = model2.predict(img, verbose=0)[0]
    disease = MODEL2_CLASSES[np.argmax(p2)]
    print(f"Model 2 = Disease: {disease}")

    # ✅ Model 3
    p3 = model3.predict(img, verbose=0)[0]
    severity = MODEL3_CLASSES[np.argmax(p3)]
    print(f"Model 3 = Severity: {severity}")

    print("\n✅ FINAL RESULT ✅")
    print(f"Disease: {disease}")
    print(f"Severity: {severity}")


In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')

    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    return b64decode(data.split(',')[1])


In [ ]:
print("📸 Capture an image with your webcam...")
photo = take_photo()

print("🔍 Running pipeline...")
pipeline_predict(photo)


📸 Capture an image with your webcam...


<IPython.core.display.Javascript object>

🔍 Running pipeline...
✅ Leaf Detected Successfully!
Model 1 = healthy

✅ FINAL RESULT: HEALTHY LEAF ✅


In [ ]:
!rm -r "/content/drive/MyDrive/sih_project/finalpipeline.ipynb"


In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/finalpipeline.ipynb" \
    "/content/drive/MyDrive/sih_project/"


In [ ]:
rm -r "/content/drive/MyDrive/sih_project/final_pipeline.h5"


In [ ]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
import tensorflow as tf

model1 = load_model("/content/drive/MyDrive/sih_project/model1_final.h5")
model2 = load_model("/content/drive/MyDrive/sih_project/model2_final_cotton_disease.h5")
model3 = load_model("/content/drive/MyDrive/sih_project/model3_final_severity.h5")


In [ ]:
def wrap_model(original_model, prefix):
    inp = Input(shape=original_model.input_shape[1:], name=f"{prefix}_input")
    out = original_model(inp)
    wrapped = Model(inputs=inp, outputs=out, name=f"{prefix}_model")
    return wrapped


In [ ]:
health_model   = wrap_model(model1, "health")
disease_model  = wrap_model(model2, "disease")
severity_model = wrap_model(model3, "severity")
# models

In [ ]:
pipeline_input = Input(shape=(224,224,3), name="pipeline_input")

o1 = health_model(pipeline_input)
o2 = disease_model(pipeline_input)
o3 = severity_model(pipeline_input)

pipeline_model = Model(
    inputs=pipeline_input,
    outputs=[o1, o2, o3],
    name="final_pipeline"
)

pipeline_model.save("/content/drive/MyDrive/sih_project/final_pipeline.h5")
print("✅ FINAL PIPELINE SAVED SUCCESSFULLY!")


✅ FINAL PIPELINE SAVED SUCCESSFULLY!
